In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("a_vsu_glucose1_201708071431.csv")

In [3]:
data.measurement_name.unique()

array(['Глюкоза', 'Билирубин общий', 'Холестерин общий',
       'Аспартатаминотрансфераза (АсАТ)', 'Аланинаминотрансфераза (АлАТ)',
       'Кортизол', 'Тестостерон свободный',
       'Антиген вируса гепатита В (HBsAg)', 'Тромбиновое время',
       'РЭА /поджелудочная железа, кишечник/',
       'Волчаночный антикоагулянт', 'Антитела к хламидии трахоматис IgA'],
      dtype=object)

In [21]:
data.head()

,patient_id,measurement_date,measurement_id,measurement_name,measurement_result
0,69,2013-04-11,27,Глюкоза,7.29
1,69,2013-04-11,28,Билирубин общий,33.40
2,69,2013-04-11,37,Холестерин общий,4.02
3,69,2013-04-11,42,Аспартатаминотрансфераза (АсАТ),30.50
4,69,2013-04-11,43,Аланинаминотрансфераза (АлАТ),47.60


In [5]:
def get_records_dict(data):
    key = 0
    d = {}
    for index, row in data.iterrows():
        value = [row.patient_id, row.measurement_date]
        if value not in d.values():
            d[key] = value
            key += 1
    return d

In [12]:
#~3 mins
records_dict = get_records_dict(data)

In [39]:
len(records_dict)

41146

In [37]:
def prepare_df(records_dict):
    measurements_dict_df = pd.DataFrame(data[['measurement_id', 'measurement_name']].drop_duplicates().values, 
                                    columns=['measurement_id', 'measurement_name'])
    general_columns = ['record_id', 'patient_id', 'measurement_date']
    measurements_columns = ['measurement_' + str(x) for x in data.measurement_id.unique()]
    columns = np.append(general_columns, measurements_columns)
    df = pd.DataFrame(columns=columns)
    
    i = 0
    for key, value in records_dict.items():
        if i % 100 == 0:
            print(i)
        i+=1
        row = pd.DataFrame([[key, value[0], value[1]]], columns=['record_id', 'patient_id', 'measurement_date'])
        df = df.append(row, ignore_index=True)[df.columns.tolist()]
    return df

In [60]:
#add elapsed time
def prepare_df_new(data):
    measurements_dict_df = pd.DataFrame(data[['measurement_id', 'measurement_name']].drop_duplicates().values, 
                                    columns=['measurement_id', 'measurement_name'])
    general_columns = ['record_id', 'patient_id', 'measurement_date']
    measurements_columns = ['measurement_' + str(x) for x in data.measurement_id.unique()]
    columns = np.append(general_columns, measurements_columns)
    df = pd.DataFrame(columns=columns)
    
    key = 0
    d = {}
    i = 0
    for index, row in data.iterrows():
        if (i + 1) % 5000 == 0:
            print(str(round(i / 137222 * 100, 1)) + '% done')
        i+=1
        value = [row.patient_id, row.measurement_date]
        if value not in d.values():
            d[key] = value
            row_data = pd.DataFrame([[key, value[0], value[1]]], 
                                    columns=['record_id', 'patient_id', 'measurement_date'])
            df = df.append(row_data, ignore_index=True)[df.columns.tolist()]
            key += 1
        measurement = 'measurement_' + str(row.measurement_id)
        df.loc[key-1][measurement] = row['measurement_result']
        
    return df

In [61]:
df = prepare_df_new(data)

3.6% done
7.3% done
10.9% done
14.6% done
18.2% done
21.9% done
25.5% done
29.1% done
32.8% done
36.4% done
40.1% done
43.7% done
47.4% done
51.0% done
54.7% done
58.3% done
61.9% done
65.6% done
69.2% done
72.9% done
76.5% done
80.2% done
83.8% done
87.4% done
91.1% done
94.7% done
98.4% done


In [63]:
df.head(20)

,record_id,patient_id,measurement_date,measurement_27,measurement_28,measurement_37,measurement_42,measurement_43,measurement_68,measurement_76,measurement_89,measurement_65,measurement_119,measurement_67,measurement_107
0,0,69,2013-04-11,7.29,33.4,4.02,30.5,47.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,69,2013-04-30,7.16,24.57,4.08,39.7,54.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,69,2013-05-24,6.65,24.95,NaN,34.5,50.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,109,2013-02-06,6.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,109,2015-06-25,6.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,178,2015-10-19,4.74,NaN,NaN,20.6,27.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,178,2015-11-26,4.88,NaN,NaN,17.5,27.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,178,2016-01-26,4.56,NaN,NaN,20.7,20.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,178,2016-03-11,4.7,12.87,5.05,34.3,72.4,726.4,0.335,NaN,NaN,NaN,NaN,NaN
9,9,217,2013-06-09,6.3,10.38,6.46,23.8,28.5,NaN,0.098,NaN,NaN,NaN,NaN,NaN


In [64]:
df.to_csv(index=False)

'record_id,patient_id,measurement_date,measurement_27,measurement_28,measurement_37,measurement_42,measurement_43,measurement_68,measurement_76,measurement_89,measurement_65,measurement_119,measurement_67,measurement_107\n0,69,2013-04-11,7.29,33.4,4.02,30.5,47.6,,,,,,,\n1,69,2013-04-30,7.16,24.57,4.08,39.7,54.4,,,,,,,\n2,69,2013-05-24,6.65,24.95,,34.5,50.4,,,,,,,\n3,109,2013-02-06,6.14,,,,,,,,,,,\n4,109,2015-06-25,6.05,,,,,,,,,,,\n5,178,2015-10-19,4.74,,,20.6,27.1,,,,,,,\n6,178,2015-11-26,4.88,,,17.5,27.7,,,,,,,\n7,178,2016-01-26,4.56,,,20.7,20.5,,,,,,,\n8,178,2016-03-11,4.7,12.87,5.05,34.3,72.4,726.4,0.335,,,,,\n9,217,2013-06-09,6.3,10.38,6.46,23.8,28.5,,0.098,,,,,\n10,217,2014-12-13,5.82,15.26,5.81,32.4,37.4,,,,,,,\n11,217,2015-03-10,5.31,16.05,4.97,16.7,19.4,,0.21899999999999997,,,,,\n12,247,2013-11-18,5.5,7.2,,25.0,34.0,,,,,,,\n13,247,2014-06-24,5.44,,6.03,25.1,34.7,,,,,,,\n14,247,2015-07-30,,,6.05,18.6,19.1,,,,,,,\n15,273,2013-05-30,6.63,9.61,,18.1,16.4,,,0.21,,,,\n16,273,2016-03-

In [ ]:
value1 = [1, "a"]
value2 = [2, "b"]
value3 = [1, "a"]
value4 = [1, "a"]
z = {}
z[0] = value1
value3 in z.values()